In [ ]:
!python3 -m pip install "scikit-learn" "pandas" "numpy" "joblib"

In [ ]:
from joblib import dump, load
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import gc

In [ ]:
!wget -O "test.parquet.gzip" "https://iitkgpacin-my.sharepoint.com/:u:/g/personal/rushilv_kgpian_iitkgp_ac_in/EXPYjAeWagtJkXj5Jq16m0MBjYsOb9GGXW2ZFMBwh0J9ZA?download=1"
!wget -O "train.parquet.gzip" "https://iitkgpacin-my.sharepoint.com/:u:/g/personal/rushilv_kgpian_iitkgp_ac_in/EbSDQP8WT9RIjIisg7PEa9UB7tuSf43J2_h3W0KgxK47ug?download=1"
!wget -O "low_imp_features.joblib" "https://iitkgpacin-my.sharepoint.com/:u:/g/personal/rushilv_kgpian_iitkgp_ac_in/EahAFwUUFahChNa0WX6C8-IBMgV0jy51QtCIAaGE3rdyXQ?download=1"

In [ ]:
pred_cols = ["Mean_BMI","Median_BMI","Unmet_Need_Rate","Under5_Mortality_Rate","Skilled_Birth_Attendant_Rate","Stunted_Rate"]

In [ ]:
def mcrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return np.average(rmse)

In [ ]:
low_imp_features = load('./low_imp_features.joblib')

In [ ]:
train = pd.read_parquet('./train.parquet.gzip')
train = train[~train.index.duplicated(keep='first')]
X = train.drop(pred_cols, axis=1)
y = train[pred_cols]

In [ ]:
del(train)
gc.collect()

In [ ]:
# drop the features with low importance and return a new dataframe
X = X.drop(columns=low_imp_features)
X

In [ ]:
test = pd.read_parquet('./test.parquet.gzip')
test = test.drop(columns=low_imp_features)

In [ ]:
model = RandomForestRegressor(n_estimators=8500, max_depth=20, max_features=0.55, random_state=42, n_jobs=-1, verbose=1)
model.fit(X, y)

In [ ]:
dump(model, './test_rf.joblib')

In [ ]:
y_pred = model.predict(test)

In [ ]:
out = pd.DataFrame(y_pred, columns=pred_cols)
out['DHSID'] = test.index
out = out[['DHSID'] + pred_cols]
out.to_csv('./new_test.csv', index=False)

In [ ]:
out